In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from armscan_env import config
from armscan_env.clustering import TissueClusters
from armscan_env.envs.state_action import ManipulatorAction
from armscan_env.util.visualizations import show_clusters
from armscan_env.volumes.slicing import (
    EulerTransform,
    create_transformed_volume,
    get_volume_slice,
)

config = config.get_config()

In [ ]:
volume = sitk.ReadImage(config.get_labels_path(1))
volume_img = sitk.GetArrayFromImage(volume)
plt.imshow(volume_img[40, :, :])
action = ManipulatorAction(rotation=(19, 0), translation=(0, 140))

o = volume.GetOrigin()
x_dash = np.arange(volume_img.shape[2])
b = volume.TransformPhysicalPointToIndex([o[0], o[1] + action.translation[1], o[2]])[1]
y_dash = x_dash * np.tan(np.deg2rad(action.rotation[0])) + b
plt.plot(x_dash, y_dash, linestyle="--", color="red")

plt.show()

In [ ]:
sliced_volume = get_volume_slice(
    action=action,
    volume=volume,
    slice_shape=(volume.GetSize()[0], volume.GetSize()[2]),
)
sliced_img = sitk.GetArrayFromImage(sliced_volume)[:, 0, :]
print(f"Slice value range: {np.min(sliced_img)} - {np.max(sliced_img)}")

slice = sliced_img
plt.imshow(slice, aspect=6)
plt.show()

In [ ]:
volume_transformation = ManipulatorAction(rotation=(19, 0), translation=(15, 15))
transformed_volume = create_transformed_volume(volume, volume_transformation)
transformed_action = EulerTransform(volume_transformation).transform_action(action)

In [ ]:
transformed_img = sitk.GetArrayFromImage(transformed_volume)
plt.imshow(transformed_img[40, :, :])

ot = transformed_volume.GetOrigin()
x_dash = np.arange(transformed_img.shape[2])
b = volume.TransformPhysicalPointToIndex([o[0], o[1] + transformed_action.translation[1], o[2]])[1]
y_dash = x_dash * np.tan(np.deg2rad(transformed_action.rotation[0])) + b
plt.plot(x_dash, y_dash, linestyle="--", color="red")

plt.show()

In [ ]:
sliced_transformed_volume = get_volume_slice(
    action=transformed_action,
    volume=transformed_volume,
    slice_shape=(volume.GetSize()[0], volume.GetSize()[2]),
)
sliced_img = sitk.GetArrayFromImage(sliced_transformed_volume)[:, 0, :]
print(f"Slice value range: {np.min(sliced_img)} - {np.max(sliced_img)}")

slice = sliced_img
plt.imshow(slice, aspect=6)
plt.show()

In [ ]:
volume_2 = sitk.ReadImage(config.get_labels_path(2))
volume_2_img = sitk.GetArrayFromImage(volume_2)
spacing = volume_2.GetSpacing()
plt.imshow(volume_2_img[51, :, :])
action_2 = ManipulatorAction(rotation=(5, 0), translation=(0, 112))

o = volume_2.GetOrigin()
x_dash = np.arange(volume_2_img.shape[2])
b = volume_2.TransformPhysicalPointToIndex([o[0], o[1] + action_2.translation[1], o[2]])[1]
y_dash = x_dash * np.tan(np.deg2rad(action_2.rotation[0])) + b
plt.plot(x_dash, y_dash, linestyle="--", color="red")

plt.show()

In [ ]:
sliced_volume_2 = get_volume_slice(
    action=action_2,
    volume=volume_2,
    slice_shape=(volume_2.GetSize()[0], volume_2.GetSize()[2]),
)
sliced_img_2 = sitk.GetArrayFromImage(sliced_volume_2)[:, 0, :]
np.save("./array", sliced_img_2)

cluster = TissueClusters.from_labelmap_slice(sliced_img_2.T)
show_clusters(cluster, sliced_img_2.T, aspect=spacing[2] / spacing[0])

plt.show()